#### preprocessing-1

In [1]:
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

C:\ProgramData\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.

In [5]:
train_data = pd.read_csv('C:/inkyun/실습파일과 교재/5.자연어처리(실습파일)/dataset/4-1.labeledTrainData.tsv', header = 0, delimiter='\t', quoting=3)

In [6]:
def preprocessing(review, stops, remove_stopwords = True):
    review_text = BeautifulSoup(review, 'html5lib').get_text()
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    words = review_text.lower().split()
    words = [w for w in words if not w in stops]
    stemmer = PorterStemmer()
    words = [stemmer.stem(w) for w in words]
    clean_review = ' '.join(words)
    return clean_review

In [7]:
stops = set(stopwords.words('english'))
clean_train_reviews = []
for review in train_data['review']:
    r = preprocessing(review, stops, remove_stopwords = True)
    clean_train_reviews.append(r)

In [8]:
clean_train_reviews[0]

'stuff go moment mj start listen music watch odd documentari watch wiz watch moonwalk mayb want get certain insight guy thought realli cool eighti mayb make mind whether guilti innoc moonwalk part biographi part featur film rememb go see cinema origin releas subtl messag mj feel toward press also obviou messag drug bad kay visual impress cours michael jackson unless remot like mj anyway go hate find bore may call mj egotist consent make movi mj fan would say made fan true realli nice actual featur film bit final start minut exclud smooth crimin sequenc joe pesci convinc psychopath power drug lord want mj dead bad beyond mj overheard plan nah joe pesci charact rant want peopl know suppli drug etc dunno mayb hate mj music lot cool thing like mj turn car robot whole speed demon sequenc also director must patienc saint came film kiddi bad sequenc usual director hate work one kid let alon whole bunch perform complex danc scene bottom line movi peopl like mj one level anoth think peopl stay 

In [9]:
clean_train_df = pd.DataFrame({'review': clean_train_reviews,'sentiment':train_data['sentiment']})

In [13]:
clean_train_df.to_csv('C:/inkyun/실습파일과 교재/5.자연어처리(실습파일)/dataset/1.train_clean.csv', index = False)

preprocessing-2

In [14]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle

In [16]:
train_data = pd.read_csv('C:/inkyun/실습파일과 교재/5.자연어처리(실습파일)/dataset/1.train_clean.csv')

In [17]:
reviews = list(train_data['review'])
sentiments = np.array(list(train_data['sentiment']))

In [18]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews)
word2idx = tokenizer.word_index
idx2word = {v:k for k, v in word2idx.items()}
text_sequences = tokenizer.texts_to_sequences(reviews)

In [20]:
word_label = []
for review, label in zip(text_sequences, sentiments):
    for w in review:
        word_label.append([w,label])

In [21]:
word_label = np.array(word_label)

In [22]:
x = np.array([np.where(word_label[:, 1] == i)[0] for i in [0, 1]])

# p(y=0)과 p(y=1)을 계산해 둔다. py[0] = p(y=0), py[1] = p(y=1)
py = np.array([(word_label[:, 1] == i).mean() for i in [0, 1]])
N = len(idx2word)

In [ ]:
mi_word = []
for i in range(1, N):
    px = (word_label[:, 0] == i).mean()
    
    mi = 0
    for y in [0, 1]:
        pxy = (word_label[x[y], 0] == i).mean()
        mi += (pxy * py[y]) * np.log(1e-8 + pxy / px)
        
    mi_word.append([mi, i])
    
    if i % 100 == 0:
        print(i, '/', N)

100 / 50597
200 / 50597
300 / 50597
400 / 50597
500 / 50597
600 / 50597
700 / 50597
800 / 50597
900 / 50597
1000 / 50597
1100 / 50597
1200 / 50597
1300 / 50597
1400 / 50597
1500 / 50597
1600 / 50597
1700 / 50597
1800 / 50597
1900 / 50597
2000 / 50597
2100 / 50597
2200 / 50597
2300 / 50597
2400 / 50597
2500 / 50597
2600 / 50597
2700 / 50597
2800 / 50597
2900 / 50597
3000 / 50597
3100 / 50597
3200 / 50597
3300 / 50597
3400 / 50597
3500 / 50597
3600 / 50597
3700 / 50597
3800 / 50597
3900 / 50597
4000 / 50597
4100 / 50597
4200 / 50597
4300 / 50597
4400 / 50597
4500 / 50597
4600 / 50597
4700 / 50597
4800 / 50597
4900 / 50597
5000 / 50597
5100 / 50597
5200 / 50597
5300 / 50597
5400 / 50597
5500 / 50597
5600 / 50597
5700 / 50597
5800 / 50597
5900 / 50597
6000 / 50597
6100 / 50597
6200 / 50597
6300 / 50597
6400 / 50597
6500 / 50597
6600 / 50597
6700 / 50597
6800 / 50597
6900 / 50597
7000 / 50597
7100 / 50597
7200 / 50597
7300 / 50597
7400 / 50597


In [ ]:
import pickle

with open('C:/Users/moon/Desktop/5.자연어처리(실습파일)/saved_data/mi_word.pickle', 'rb') as f:
    mi_word = pickle.load(f)

In [29]:
# mi_word 리스트를 내림차순으로 정렬한다.
mi_word.sort(reverse = True)

# MI 상위 20개 단어를 확인해 본다. Pelaez의 문서 내용과 일치하는지 확인한다.
print([idx2word[y] for x, y in mi_word[:20]])

# MI 상위 50% 단어로 vocabulrary를 생성한다.
n = int(len(mi_word) * 0.5)
word2idx2 = {idx2word[y]:(i+1) for i, [x, y] in enumerate(mi_word[:n])}
idx2word2 = {v:k for k, v in word2idx.items()}

# MI 기반 vocabulary를 이용하여 리뷰 데이터를 다시 만든다. OOV는 제거한다.
new_review = []
for review in reviews:
    r = []
    for w in review.split():
        if w in word2idx2: # OOV 제거
            r.append(w)
    new_review.append(r)

['bad', 'fun', 'wast', 'great', 'camera', 'think', 'movi', 'poor', 'featur', 'although', 'evil', 'recommend', 'impress', 'may', 'new', 'littl', 'portray', 'exist', 'kill', 'stori']


In [30]:
print(list(word2idx)[:20])

['movi', 'film', 'one', 'like', 'time', 'good', 'make', 'charact', 'get', 'see', 'watch', 'stori', 'even', 'would', 'realli', 'well', 'scene', 'look', 'show', 'much']


In [31]:
model_name = 'dataset/3.400features.doc2vec'
moedl_saved = True
EMB_SIZE = 400

In [36]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(reviews)]
model = Doc2Vec(vector_size = EMB_SIZE, alpth=0.005, min_alpha = 0.0001, min_count = 1, workers=4, dm=1)
model.build_vocab(documents)
model.train(documents, total_examples = len(reviews), epochs = 500)
model.save(model_name)

KeyboardInterrupt: 

In [ ]:
pvdm = np.array([model.docvecs[i] for i in range(len(reviews))])

In [ ]:
sentences = []
for review in reviews:
    sentences.append(' '.join(review))

In [ ]:
vectorizer = TfidfVectorizer(min_df = 1, analyzer = 'word', sublinear_tf = True, ngram_range=(1,1), max_features=EMB_SIZE)
tfidf = vectorizer.fit_trainsform(sentences).toarray()

In [ ]:
x1_train, x1_test, x2_train, x2_test, y_train, y_test = \
    train_test_split(pvdm, tfidf, y, test_size=0.2)